In [ ]:
import tifffile
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os
import UTXCTregister as reg
import openpyxl
from pathlib import Path
import sys
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the parent directory to sys.path
sys.path.append(parent_dir)
from AutoUTAligner import autoutaligner as alg

In [ ]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

In [ ]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

In [ ]:
database.columns

In [ ]:
ids = database['Id'].values

frontals  = database['Frontal 90 Right'].values

amps = database['UT aligned amp'].values

rfs = database['UT aligned'].values

rfs_og = database['UT'].values

rfs_og_lente = database['UT lente'].values

amps_lente = database['UT aligned amp lente'].values

rfs_lente = database['UT aligned lente'].values

registered_uts = database['UT registered'].values

registered_uts_lente = database['UT registered lente'].values

registration_parameters = database['Registration Parameters'].values

registration_parameters_lente = database['Registration Parameters lente'].values

In [ ]:
for i in range(len(ids)):

    #utvsxct register
    if registration_parameters[i] == None: #if is not already registered

        try:

            print(f'Registering {ids[i]}')

            try:
                ut_file = str(rfs_og[i])
            except:
                print('No UT file')
                continue

            ut_output_file = Path(ut_file[:-4] + '_registered.tif')

            #create a folder named aux_registration in the same folder as the XCT file
            xct_file = Path(frontals[i])
            output_folder = xct_file.parent / 'aux_registration'
            output_folder.mkdir(parents=True, exist_ok=True)

            rf = tifffile.imread(rfs_og[i])
            amp = alg.hillbert_transform(rf)
            gate = alg.auto_gate(amp)
            # Align the amplitude and the volume based on the maximum value in the gated range
            amp, _ = alg.double_align(amp,rf, gate)
            amp = np.swapaxes(amp, 0, 1)
            amp = np.swapaxes(amp, 1, 2)

            try:
                xct = tifffile.imread(frontals[i])
                xct = np.swapaxes(xct, 0, 1)
                xct = np.swapaxes(xct, 1, 2)
            except:
                print('XCT file error')
                continue

            parameters,ut_centers,xct_centers, transformed = reg.main_2(amp,xct)

            registration_parameters[i] = parameters

            # Save the aux files
            aux_output_file = output_folder / 'centers.tif'
            tifffile.imwrite(aux_output_file, ut_centers)
            aux_output_file = output_folder / 'centers_xct.tif'
            tifffile.imwrite(aux_output_file, xct_centers)
            aux_output_file = output_folder / 'transformed_centers_xct.tif'
            tifffile.imwrite(aux_output_file, transformed)

        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)
    
    if registration_parameters_lente[i] == None:

        try:

            print(f'Registering {ids[i]} lens')

            try:
                ut_file = str(rfs_og_lente[i])
            except:
                print('No UT file')
                continue

            ut_output_file = Path(ut_file[:-4] + '_registered.tif')

            #create a folder named aux_registration in the same folder as the XCT file
            xct_file = Path(frontals[i])
            output_folder = xct_file.parent / 'aux_registration'
            output_folder.mkdir(parents=True, exist_ok=True)

            rf = tifffile.imread(rfs_og_lente[i])
            amp = alg.hillbert_transform(rf)
            gate = alg.auto_gate(amp)
            # Align the amplitude and the volume based on the maximum value in the gated range
            amp, _ = alg.double_align(amp,rf, gate)
            amp = np.swapaxes(amp, 0, 1)
            amp = np.swapaxes(amp, 1, 2)

            try:
                xct = tifffile.imread(frontals[i])
                xct = np.swapaxes(xct, 0, 1)
                xct = np.swapaxes(xct, 1, 2)
            except:
                print('XCT file error')
                continue

            parameters,ut_centers,xct_centers, transformed = reg.main_2(amp,xct)

            registration_parameters_lente[i] = parameters

            # Save the aux files
            aux_output_file = output_folder / 'centers.tif'
            tifffile.imwrite(aux_output_file, ut_centers)
            aux_output_file = output_folder / 'centers_xct.tif'
            tifffile.imwrite(aux_output_file, xct_centers)
            aux_output_file = output_folder / 'transformed_centers_xct.tif'
            tifffile.imwrite(aux_output_file, transformed)
        
        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)

In [ ]:
#update database

database['UT registered'] = registered_uts

database['UT registered lente'] = registered_uts_lente

database['Registration Parameters'] = registration_parameters

database['Registration Parameters lente'] = registration_parameters_lente

In [ ]:
database.to_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv', index=False)

# Loading parameters

In [ ]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

parameters = database['Registration Parameters'].values

a = parameters[0]

a

In [ ]:
def to_matrix(string):

    matrix1 = float(string[2:17])

    matrix2 = float(string[17:33])

    matrix3 = float(string[33:49])

    matrix4 = float(string[53:68])

    matrix5 = float(string[68:84])

    matrix6 = float(string[84:100])

    matrix7 = float(string[105:121])

    matrix8 = float(string[121:137])

    matrix9 = float(string[137:151])

    matrix = np.array([[matrix1,matrix2,matrix3],[matrix4,matrix5,matrix6],[matrix7,matrix8,matrix9]])

    return matrix

In [ ]:
to_matrix(a)